In [1]:
# libraries
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.integrate import simps
import scipy.constants as cte
from scipy.sparse import diags
from scipy.linalg import inv
from scipy.fftpack import fft, ifft, fftfreq

In [2]:
# grandezas de interesse em unidades atomicas
au_l = cte.value('atomic unit of length')
au_t = cte.value('atomic unit of time')
au_e = cte.value('atomic unit of energy')

# outras relacoes de interesse
ev = cte.value('electron volt')
au2ang = au_l / 1e-10
au2ev = au_e / ev

In [ ]:
# salva os valores onde o algoritmo converge
par_convergentes = []
for L in np.linspace(100,1000,7):
    # unidades do problema
    E_0 = 150.0 # eV
    #L = 100.0 # angstron
    sigma_x = 1.0 # angstron
    x_0 = -20.0 # angstron
    dt = dt_0 = 1e-17 # s

    # unidades atomicas
    E_0_au = E_0 / au2ev
    L_au = L / au2ang
    sigma_x_au = sigma_x / au2ang
    x_0_au = x_0 / au2ang
    dt_au = dt / au_t
    k_0_au = np.sqrt(2 * E_0_au)

    # divisor esperto (?)
    de = lambda x: 2 if int((x/(10**(int(np.log10(x))-1)))%2) == 0 else 5

    for N in [2**n for n in range(9,12)]:
        if float(L) / float(N) > 0.5:
            continue
            
        dt = dt_0

        # malha espacial
        x_au = np.linspace(-L_au/2, L_au/2, N)
        dx_au = x_au[1] - x_au[0]

        k_au = fftfreq(N, d=dx_au)

        while True:
            #dt /= 10
            dt /= de(dt)
            dt_au = dt / au_t

            # split step
            exp_v2 = lambda t: 1.0
            exp_t = np.exp(- 0.5j * (2 * np.pi * k_au) ** 2 * dt_au)
            evolve_real = lambda psi, t: exp_v2(t) * ifft(exp_t * fft(exp_v2(t) * psi))

            # pacote de onda
            PN = 1/(2*np.pi*sigma_x_au**2)**(1/4)
            psi = PN*np.exp(1j*k_0_au*x_au-(x_au-x_0_au)**2/(4*sigma_x_au**2))
            A0 = (simps(np.conjugate(psi)*psi,x_au)).real
            x_f_au = x_0_au 

            funcionou = True
            contador = 0
            norma = 100
            skewness = 0

            while x_f_au < -x_0_au:
                try:
                    psi = evolve_real(psi, 0)
                    contador += 1
                    #if contador % 10 == 0:
                    psis = np.conjugate(psi)
                    A = (simps(psis*psi,x_au)).real
                    norma = 100 * A / A0
                    if np.abs(norma - 100) > 5:
                        funcionou = False
                        break

                    x_f_au = xm1 = (simps(psis* x_au * psi,x_au)).real / A
                    xm2 = (simps(psis* x_au**2 * psi,x_au)).real / A
                    xm3 = (simps(psis* x_au**3 * psi,x_au)).real / A

                    sigma = np.sqrt(np.abs(xm2 - xm1**2))
                    skewness = gamma = (xm3 - 3*xm1*sigma**2-xm1**3)/sigma**3
                    if np.abs(gamma) > 0.5:
                        funcionou = False
                        break
                except:
                    funcionou = False
                    break

            parametros = (L, N, dt, norma, skewness, contador)
            if funcionou:
                par_convergentes.append(parametros)
                break
            print("Estouro: L = {}, N = {}, dt={:.2e} s, A = {:.2f}, G = {:.2f}, contador = {}".format(*parametros))
    print(par_convergentes)
print(par_convergentes)

[(100.0, 512, 5e-18, 100.00000000000018, 1.4153561677050928e-13, 111), (100.0, 1024, 5e-18, 100.00000000000031, 1.4153561677049257e-13, 111), (100.0, 2048, 5e-18, 100.00000000000003, 5.6614246708195602e-14, 111)]
Estouro: L = 250.0, N = 512, dt=5.00e-18 s, A = 99.95, G = -0.50, contador = 247
Estouro: L = 250.0, N = 512, dt=1.00e-18 s, A = 99.95, G = -0.50, contador = 1234
Estouro: L = 250.0, N = 512, dt=5.00e-19 s, A = 99.95, G = -0.50, contador = 2468
Estouro: L = 250.0, N = 512, dt=1.00e-19 s, A = 99.95, G = -0.50, contador = 12339
Estouro: L = 250.0, N = 512, dt=5.00e-20 s, A = 99.95, G = -0.50, contador = 24677
Estouro: L = 250.0, N = 512, dt=1.00e-20 s, A = 99.95, G = -0.50, contador = 123383
Estouro: L = 250.0, N = 512, dt=5.00e-21 s, A = 99.95, G = -0.50, contador = 246765
Estouro: L = 250.0, N = 512, dt=1.00e-21 s, A = 99.95, G = -0.50, contador = 1233824


In [4]:
np.ones(2)+7

array([ 8.,  8.])

In [2]:
np.abs(-1)

1